# Imports

In [3]:
import selenium
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#import numpy as np
import csv

In [4]:
import json
import os.path as osp
import platform
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.proxy import Proxy, ProxyType


In [5]:
import re

# Browser

In [6]:
browser = webdriver.Chrome(executable_path="chromedriver")
browser.maximize_window()
sleep(1)

# Extract items function

In [19]:
def extract_item_info_mxb(product_link):
    browser.get(product_link)
    sleep(10) # slow to sign in
    
    # scroll all page by Chiu
    total_height = int(browser.execute_script("return document.body.scrollHeight"))
    for i in range(1, total_height, 100):
        browser.execute_script("window.scrollTo(0, {});".format(i))
    
    # parse html doc by Chiu
    soup = BeautifulSoup(browser.page_source, "html.parser")

    # initialize a dictionary to store information of products
    item_info = {
            "insurance": "", # insurance type (str)
            "shock deal": "", # Deal Soc: Mua De Nhan Qua/Mua Kem Deal Soc/... (str)
            "gift min price": 0, # minimum requirement of paying to receive gift, none: -1 (float)
            "flash sale promo soon": -1, # 1 for yes, 0 for no (0/1)
            "cheap tag": "", # Gi Cung Re (gia tot nhat so voi san pham cung loai tren Shopee)
            "price" : 0,
            "price min": 0,
            "price max": 0,
            "raw discount": -1, # x if discount x%
            "price before discount": 0,
            "price before discount min": 0,
            "price before discount max": 0
        }
    
    # extract insurance type
    insurance_tag = soup.find_all("div", {"class": "flex rY0UiC"}) 
    if insurance_tag:
        for n in insurance_tag:
            insurance_label = n.find("div", {"class": "flex items-center"})
            if insurance_label is not None:
                insurance = n.find('div', class_=None, style=None).text
                break
            else:
                insurance = ""
    else:
        insurance = ""
    item_info["insurance"] = insurance

    # extract shock deal type
    shock_deal_tag = soup.find("div", {"class": "q9zbL+"})
    if shock_deal_tag:
        shock_deal_label = shock_deal_tag.text
    else:
        shock_deal_label = ""
    item_info["shock deal"] = shock_deal_label

    # extract gift min price
    gift_min_price_tag = soup.find("div", {"class" : "hp69aN"})
    if gift_min_price_tag:
        gift_min_price_label = gift_min_price_tag.text
        gift_min_price_label = float(gift_min_price_label.split()[1][1:]) * 1000
    else:
        gift_min_price_label = -1
    item_info["gift min price"] = gift_min_price_label
    
    # extract flash sale or promotion soon
    flash_sale_soon_tag = soup.find("div", {'class': "aH5TuA"})
    if flash_sale_soon_tag:
        flash_sale_soon_label = 1
    else:
        flash_sale_soon_label = 0
    item_info["flash sale promo soon"] = flash_sale_soon_label

    # extract cheap tag
    cheap_tag = soup.find("div", {"class" : "u86SRJ"})
    if cheap_tag:
        cheap_label = cheap_tag.text
    else:
        cheap_label = ""
    item_info["cheap tag"] = cheap_label

    # extract price, price min, price max: 
    #  if have price range (min - max): price = price min
    #  if NOT have price range (min - max): price min = price max = price
    price_tag = soup.find("div", {"class" : "pqTWkA"})
    if price_tag:
        price_label = price_tag.text
        price_original = float(price_label.split()[0][1:]) * 1000
        if len(price_label.split()) > 1:
            price_min = price_original 
            price_max = float(price_label.split()[2][1:]) * 1000
        else:
            price_min = price_original 
            price_max = price_original 
    item_info["price"] = price_original 
    item_info["price max"] = price_max
    item_info["price min"] = price_min

    # extract raw discount
    raw_discount_tag = soup.find("div", {"class": "_0voski"})
    if raw_discount_tag:
        raw_discount_label = raw_discount_tag.text
        raw_discount_label = float(raw_discount_label.split()[0][:-1])
    else:
        raw_discount_label = 0
    item_info["raw discount"] = raw_discount_label


    # extract price before discount, min and max price before discount
    #  if have price range (min - max): price before discount = price before discount min
    #  if NOT have price range (min - max): price before discount min  = price before discount  max = price before discount 
    price_before_discount_tag = soup.find("div", {"class" : "Y3DvsN"})
    if price_before_discount_tag:
        price_before_discount_label = price_before_discount_tag.text
        price_before_discount_original = float(price_before_discount_label.split()[0][1:]) * 1000
        if len(price_before_discount_label.split()) > 1:
            price_before_discount_min = price_before_discount_original 
            price_before_discount_max = float(price_before_discount_label.split()[2][1:]) * 1000
        else:
            price_before_discount_min = price_before_discount_original 
            price_before_discount_max = price_before_discount_original 
    else:
        price_before_discount_original = -1
        price_before_discount_min = -1
        price_before_discount_max = -1
    item_info["price before discount"] = price_before_discount_original 
    item_info["price before discount max"] = price_before_discount_max
    item_info["price before discount min"] = price_before_discount_min
        
    return item_info

# Demo extract 4 links to check

In [8]:
test_links = ["https://shopee.vn/Ão-thun-nam-ná»¯-unisex-tay-lá»¡-Ã¡o-phÃ´ng-cotton-from-rá»™ng-AD69-Oversize-i.476666030.11817415931?sp_atk=5f57cf44-7dbb-4661-a466-31b9aee97886&xptdk=5f57cf44-7dbb-4661-a466-31b9aee97886",
            "https://shopee.vn/N%E1%BA%BFn-Th%C6%A1m-Trang-Tr%C3%AD-Cao-C%E1%BA%A5p-T%E1%BB%B1-Nhi%C3%AAn-Aroma-Story-Nhi%E1%BB%81u-H%C6%B0%C6%A1ng-Size-200g-i.434961385.12853667421",
            "https://shopee.vn/%C3%81o-thun-c%E1%BB%95-tr%C3%B2n-nam-n%E1%BB%AF-SSSTUTTER-d%C3%A1ng-r%E1%BB%99ng-in-ch%E1%BB%AF-n%E1%BB%95i-con-th%E1%BB%8F-RABBIT-TEE-i.179042207.22541012130?xptdk=86a2d847-e7fd-4ee5-8cd9-fffb8f2e4ac2",
            "https://shopee.vn/Qu%E1%BA%A7n-Jeans-Nam-BOO-D%C3%A1ng-R%E1%BB%99ng-Th%C3%AAu-Logo-BOOEST03-i.17271526.19666885941"
]
# link 1: ao thun nam nu unisex oversize
# link 2: nen thom
# link 3: ao thun co tron nam nu ssstuter
# link 4: quan jeans name BOO dang rong theu logo

In [9]:
for i, link in enumerate(test_links):
    print("Product {}".format(i + 1))
    print(extract_item_info_mxb(link))
    print("\n")


Product 1
{'insurance': 'Bảo hiểm Thời trang', 'shock deal': '', 'gift min price': -1, 'flash sale promo soon': 0, 'cheap tag': 'Gì Cũng Rẻ', 'price': 12000.0, 'price min': 12000.0, 'price max': 59000.0, 'raw discount': 54.0, 'price before discount': 26000.0, 'price before discount min': 26000.0, 'price before discount max': 122000.0}


Product 2
{'insurance': '', 'shock deal': 'Mua để nhận quà', 'gift min price': 350000.0, 'flash sale promo soon': 1, 'cheap tag': 'Gì Cũng Rẻ', 'price': 173000.0, 'price min': 173000.0, 'price max': 355000.0, 'raw discount': 6.0, 'price before discount': 200000.0, 'price before discount min': 200000.0, 'price before discount max': 485000.0}


Product 3
{'insurance': 'Bảo hiểm Thời trang', 'shock deal': 'Mua để nhận quà', 'gift min price': 599000.0, 'flash sale promo soon': 0, 'cheap tag': '', 'price': 299000.0, 'price min': 299000.0, 'price max': 299000.0, 'raw discount': 21.0, 'price before discount': 379000.0, 'price before discount min': 379000.0, 'p

# Extract information from 75 products

## jeans product

In [11]:
jean_products = pd.read_csv("./products/jeans.csv")
jean_products.head()

,Unnamed: 0,name,price,quantity_sold,shop_name,url
0,0,Quần Jean Xám Mỏng Màu Xám Khói Lưng Cao Ống R...,160.072,Đã bán 931,NaN,https://shopee.vn/Quần-Jean-Xám-Mỏng-Màu-Xám-K...
1,1,Quần jeans lưng cao ống rộng màu RETRO,139.000,"Đã bán 4,6k",NaN,https://shopee.vn/Quần-jeans-lưng-cao-ống-rộng...
2,2,"Quần jean nam nữ unisex dáng suông rộng, quần ...",99.000,"Đã bán 29,9k",NaN,https://shopee.vn/Quần-jean-nam-nữ-unisex-dáng...
3,3,Daduhey Quần jeans ống rộng eo cao thời trang ...,142.000,"Đã bán 16,9k",NaN,https://shopee.vn/Daduhey-Quần-jeans-ống-rộng-...
4,4,"Quần jean nam đen ống suông rộng CẠP CAO, Quầ...",145.000,"Đã bán 12,7k",NaN,https://shopee.vn/Quần-jean-nam-đen-ống-suông...


In [13]:
jean_products_25_links =  jean_products["url"][:25]


## t shirt products

In [14]:
tshirt_products = pd.read_csv("./products/t_shirt.csv")
tshirt_products_25_links = tshirt_products["url"][:25]
tshirt_products_25_links

0     https://shopee.vn/Áo-thun-áo-phông-tay-lỡ-in-h...
1     https://shopee.vn/Áo-Phông-Form-rộng-Wioops-Áo...
2     https://shopee.vn/Áo-thun-nữ-áo-phông-nữ-tay-n...
3     https://shopee.vn/-Rẻ-vô-địch-Áo-thun-cotton-t...
4     https://shopee.vn/-RẺ-VÔ-ĐỊCH-Áo-Thun-Tổ-Ong-L...
5     https://shopee.vn/Áo-thun-nữ-áo-phông-nữ-tay-n...
6     https://shopee.vn/Áo-thun-nam-nữ-unisex-tay-lỡ...
7     https://shopee.vn/-Rẻ-Vô-Địch-Áo-thun-cotton-t...
8     https://shopee.vn/Áo-thun-tay-lỡ-Tim-Khói-Chữ-...
9     https://shopee.vn/Áo-thun-tay-lỡ-số-11-phối-nh...
10    https://shopee.vn/Áo-Thun-I-LOTSO-FIDE-phông-t...
11    https://shopee.vn/-FREESHIP-ĐƠN-TỪ-50K-Áo-Thun...
12    https://shopee.vn/Áo-Phông-Nam-Nữ-Louboutin-Áo...
13    https://shopee.vn/(SALE)Áo-phông-hàng-tặng-mẫu...
14    https://shopee.vn/Áo-Thun-Tay-Lỡ-Unisex-BUNNY-...
15    https://shopee.vn/Áo-phông-tay-lỡ-form-rộng-na...
16    https://shopee.vn/Áo-thun-tay-lỡ-1989-chữ-hồng...
17    https://shopee.vn/Áo-thun-unisex-nam-nữ-fo

## overcoat products

In [15]:
coat_products = pd.read_csv("./products/overcoat.csv")
coat_products_25_links = coat_products["url"][:25]
coat_products_25_links

0     https://shopee.vn/Áo-khoác-dù-nam-chất-liệu-dù...
1     https://shopee.vn/áo-khoác-hoodie-NAM-Áo-Kho...
2     https://shopee.vn/Áo-khoác-hoodie-zip-khóa-kéo...
3     https://shopee.vn/Áo-Khoác-Nam-áo-khoác-dù-nam...
4     https://shopee.vn/Áo-Khoác-Local-Brand-Jacket-...
5     https://shopee.vn/Áo-Khoác-Teelab-Có-Mũ-Áo-gió...
6     https://shopee.vn/Áo-Khoác-Áo-Gió-TEELAB-Phối-...
7     https://shopee.vn/ÁO-KHOÁC-UNIXEX-FROM-RỘNG-CÁ...
8     https://shopee.vn/Áo-KHOÁC-Hoodie-Xanh-Rêu-NEV...
9     https://shopee.vn/Áo-KHOÁC-Hoodie-CLO-Ulzzang-...
10    https://shopee.vn/Áo-KHOÁC-NÚT-và-Ulzzang-Unis...
11    https://shopee.vn/ao-hoodies-nam-nữ-nỉ-bông-fo...
12    https://shopee.vn/Áo-Khoác-Bomber-Teddy-Nam-Nữ...
13    https://shopee.vn/Áo-Khoác-Áo-Gió-TEELAB-Unise...
14    https://shopee.vn/-FREESHIP-Áo-Khoác-HOODIE-Na...
15    https://shopee.vn/Áo-Khoác-Nỉ-Hoodie-Nam-Nữ-Un...
16    https://shopee.vn/Áo-khoác-thể-thao-cổ-đứng-dá...
17    https://shopee.vn/-HOT-TREND-ÁO-KHOÁC-KAKI

## extract information from these 75 products above

In [20]:
lst_of_dict_products = []
for kind in [jean_products_25_links, tshirt_products_25_links, coat_products_25_links]:
    print("New kind of products")
    for i, link in enumerate(kind):
        print("Product {}".format(i + 1))
        lst_of_dict_products.append(extract_item_info_mxb(link))
    print("\n")
    
print(len(lst_of_dict_products))
lst_of_dict_products


New kind of products
Product 1
Product 2
Product 3
Product 4
Product 5
Product 6
Product 7
Product 8
Product 9
Product 10
Product 11
Product 12
Product 13
Product 14
Product 15
Product 16
Product 17
Product 18
Product 19
Product 20
Product 21
Product 22
Product 23
Product 24
Product 25


New kind of products
Product 1


UnboundLocalError: local variable 'price_original' referenced before assignment

**Error do có sản phẩm trong tshirt bị xóa không còn tìm thấy nữa**

In [21]:
print(len(lst_of_dict_products))
lst_of_dict_products

25


[{'insurance': 'Bảo hiểm Thời trang',
  'shock deal': '',
  'gift min price': -1,
  'flash sale promo soon': 0,
  'cheap tag': '',
  'price': 215000.0,
  'price min': 215000.0,
  'price max': 215000.0,
  'raw discount': 24.0,
  'price before discount': 281429.0,
  'price before discount min': 281429.0,
  'price before discount max': 281429.0},
 {'insurance': 'Bảo hiểm Thời trang',
  'shock deal': '',
  'gift min price': -1,
  'flash sale promo soon': 1,
  'cheap tag': 'Gì Cũng Rẻ',
  'price': 126600.0,
  'price min': 126600.0,
  'price max': 149000.0,
  'raw discount': 49.0,
  'price before discount': 250000.0,
  'price before discount min': 250000.0,
  'price before discount max': 250000.0},
 {'insurance': 'Bảo hiểm Thời trang',
  'shock deal': 'Mua để nhận quà',
  'gift min price': 50000.0,
  'flash sale promo soon': 0,
  'cheap tag': 'Gì Cũng Rẻ',
  'price': 138000.0,
  'price min': 138000.0,
  'price max': 138000.0,
  'raw discount': 30.0,
  'price before discount': 198000.0,
  'pr

**Tiếp tục crawl nốt sản phẩm trong overcoat vậy**

In [22]:
for kind in [coat_products_25_links]:
    print("New kind of products")
    for i, link in enumerate(kind):
        print("Product {}".format(i + 1))
        lst_of_dict_products.append(extract_item_info_mxb(link))
    print("\n")

New kind of products
Product 1
Product 2
Product 3
Product 4
Product 5
Product 6
Product 7
Product 8
Product 9
Product 10
Product 11
Product 12
Product 13
Product 14
Product 15
Product 16
Product 17
Product 18
Product 19
Product 20
Product 21
Product 22
Product 23
Product 24


UnboundLocalError: local variable 'price_original' referenced before assignment

**Error do có sản phẩm trong áo khoác bị xóa không còn tìm thấy nữa -> Cần khắc phục trong code soon**

In [23]:
len(lst_of_dict_products)

48

In [24]:
lst_of_dict_products

[{'insurance': 'Bảo hiểm Thời trang',
  'shock deal': '',
  'gift min price': -1,
  'flash sale promo soon': 0,
  'cheap tag': '',
  'price': 215000.0,
  'price min': 215000.0,
  'price max': 215000.0,
  'raw discount': 24.0,
  'price before discount': 281429.0,
  'price before discount min': 281429.0,
  'price before discount max': 281429.0},
 {'insurance': 'Bảo hiểm Thời trang',
  'shock deal': '',
  'gift min price': -1,
  'flash sale promo soon': 1,
  'cheap tag': 'Gì Cũng Rẻ',
  'price': 126600.0,
  'price min': 126600.0,
  'price max': 149000.0,
  'raw discount': 49.0,
  'price before discount': 250000.0,
  'price before discount min': 250000.0,
  'price before discount max': 250000.0},
 {'insurance': 'Bảo hiểm Thời trang',
  'shock deal': 'Mua để nhận quà',
  'gift min price': 50000.0,
  'flash sale promo soon': 0,
  'cheap tag': 'Gì Cũng Rẻ',
  'price': 138000.0,
  'price min': 138000.0,
  'price max': 138000.0,
  'raw discount': 30.0,
  'price before discount': 198000.0,
  'pr

In [26]:
df = pd.DataFrame(lst_of_dict_products)
print(df.shape)
df.head(10)

(48, 12)


,insurance,shock deal,gift min price,flash sale promo soon,cheap tag,price,price min,price max,raw discount,price before discount,price before discount min,price before discount max
0,Bảo hiểm Thời trang,,-1.0,0,,215000.0,215000.0,215000.0,24.0,281429.0,281429.0,281429.0
1,Bảo hiểm Thời trang,,-1.0,1,Gì Cũng Rẻ,126600.0,126600.0,149000.0,49.0,250000.0,250000.0,250000.0
2,Bảo hiểm Thời trang,Mua để nhận quà,50000.0,0,Gì Cũng Rẻ,138000.0,138000.0,138000.0,30.0,198000.0,198000.0,249000.0
3,Bảo hiểm Thời trang,,-1.0,0,,136000.0,136000.0,136000.0,43.0,237000.0,237000.0,237000.0
4,Bảo hiểm Thời trang,,-1.0,0,Gì Cũng Rẻ,145000.0,145000.0,145000.0,44.0,260000.0,260000.0,260000.0
5,Bảo hiểm Thời trang,,-1.0,0,,170000.0,170000.0,210000.0,31.0,245412.0,245412.0,295794.0
6,Bảo hiểm Thời trang,,-1.0,0,,241970.0,241970.0,241970.0,0.0,-1.0,-1.0,-1.0
7,Bảo hiểm Thời trang,,-1.0,0,,88000.0,88000.0,110000.0,45.0,160000.0,160000.0,200000.0
8,Bảo hiểm Thời trang,,-1.0,0,Gì Cũng Rẻ,139000.0,139000.0,139000.0,26.0,189000.0,189000.0,189000.0
9,Bảo hiểm Thời trang,Mua để nhận quà,300000.0,0,Gì Cũng Rẻ,148000.0,148000.0,219000.0,56.0,335500.0,335500.0,496800.0


In [28]:
df.to_csv("demo_48_products_mxb.csv", encoding='utf-8')